# I. Giới thiệu chung
## 1. Giới thiệu bài toán
<span style="font-size: 18px">
    Ta có thể thấy protein đóng vai trò thiết yếu trong hầu như tất các quá trình của tế bào. Thông thường, nhiều protein kết hợp với nhau tại một vị trí cụ thể để thực hiện 1 nhiệm vụ, kết quả chính xác của nhiệm vụ này phụ thuộc vào loại protein nào có mặt. Như bạn thấy, sự phân bố của protein ở các tế bào khác nhau là khác nhau, làm phát sinh sự không đồng nhất về chức năng giữa các tế bào. Việc tìm ra những điểm khác biệt như vậy và lý do tại sao có sự phân bố khác nhau của protein giữa các tế bào có cùng chức năng là điều quan trọng để hiểu được cách thức hoạt động của các tế bào, tìm ra cách các căn bệnh phát triển, sau đó tìm ra các phương pháp điều trị tốt hơn cho những bệnh đó.
</span>

## 2. Dữ liệu
<b style="font-size: 18px">Human Protein Atlas - Single Cell Classification</b>
<br>
<span style="font-size: 18px">Bộ dữ liệu có 3 loại ảnh PNG bao gồm các kích thước:</span>
    
<ul style="font-size: 18px">
    <li>1728x1728</li>
    <li>2048x2048</li>
    <li>3072x3072</li>
</ul>

<span style="font-size: 18px">
    Tất cả đều được chụp bằng kính hiển vi đồng tiêu (Confocal Microscopy)
    <br>
    Mỗi ảnh được chụp trên 4 kênh → 4 ảnh:
<br>
</span>
<ul style="font-size: 18px">
    <li><code>red:</code> microtubule channels (<a href="https://vi.wikipedia.org/wiki/Vi_ống">Vi ống</a>)</li>
    <li><code>blue:</code> nuclei channels (<a href="https://vi.wikipedia.org/wiki/Nhân_tế_bào">Nhân tế bào</a>)</li>
    <li><code>yellow:</code> Endoplasmic Reticulum (ER) channels (<a href="https://vi.wikipedia.org/wiki/Mạng_lưới_nội_chất">Lưới nội chất</a>)</li>
    <li><code>green:</code> protein of interest (<a href="https://www.google.com/search?q=protein+of+interest&oq=protein+of+interest&aqs=edge..69i57j0l3j0i22i30l3.760j0j1&sourceid=chrome&ie=UTF-8">???</a>)</li>
</ul>

## 3. Nhiệm vụ
<span style="font-size: 18px">
    Có tất cả 19 nhãn cần phân lớp, 18 nhãn cho 18 loại tế bào và 1 nhãn cho những tế bào không xác định được cụ thể. 
    
    
</span>

<pre style="font-size: 18px">
0.  Nucleoplasm  
1.  Nuclear membrane   
2.  Nucleoli   
3.  Nucleoli fibrillar center   
4.  Nuclear speckles   
5.  Nuclear bodies   
6.  Endoplasmic reticulum   
7.  Golgi apparatus   
8.  Intermediate filaments  
9.  Actin filaments  
10.  Microtubules      
11.  Mitotic spindle   
12.  Centrosome   
13.  Plasma membrane   
14.  Mitochondria   
15.  Aggresome   
16.  Cytosol   
17.  Vesicles and punctate cytosolic patterns   
18.  Negative  
</pre>

<span style="font-size: 18px">
    Tất cả các ảnh được thể hiện bằng 4 bộ lọc:
</span>
<ul style="font-size: 18px">
    <li>Bộ lọc màu xanh(<code>green</code>) được sử dụng để dự đoán và gán nhãn</li>
    <li>3 bộ lọc còn lại(<code>red, blue, yellow</code>) được sử dụng để làm tài liệu tham khảo, cho thấy sự tương quan của tế bào giữa các bộ lọc</li>
</ul>
    
### Nhiệm vụ chính
<span style="font-size: 18px">
Với mỗi ảnh chúng ta đã được cho sẵn các nhãn của các tế bào xuất hiện trong ảnh.
<br><b> Nhiệm vụ của chúng ta là segment ra các tế bào và gán nhãn cho các tế bào đó.</b>
</span>

# II. Thực hiện

In [ ]:
!pip install https://github.com/CellProfiling/HPA-Cell-Segmentation/archive/master.zip

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

from sklearn.preprocessing import MultiLabelBinarizer

from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
DATA_DIR = "/kaggle/input/hpa-single-cell-image-classification"
train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))

colours = ['_red.png', '_blue.png', '_yellow.png', '_green.png']
TRAIN = '../input/hpa-single-cell-image-classification/train'
paths = [[os.path.join(TRAIN, train.iloc[idx,0])+ colour for colour in colours] for idx in range(len(train))]

In [ ]:
LABELS= {
    0: "Nucleoplasm",
    1: "Nuclear membrane",
    2: "Nucleoli",
    3: "Nucleoli fibrillar center",
    4: "Nuclear speckles",
    5: "Nuclear bodies",
    6: "Endoplasmic reticulum",
    7: "Golgi apparatus",
    8: "Intermediate filaments",
    9: "Actin filaments",
    10: "Microtubules",
    11: "Mitotic spindle",
    12: "Centrosome",
    13: "Plasma membrane",
    14: "Mitochondria",
    15: "Aggresome",
    16: "Cytosol",
    17: "Vesicles and punctate cytosolic patterns",
    18: "Negative"
}

## Data Visualize

In [ ]:
train

In [ ]:
train_csv = train.copy()
train_csv['Label'] = train_csv['Label'].apply(lambda x: list(map(int,x.split("|"))))
mlb = MultiLabelBinarizer()
train_csv[list(range(19))] = mlb.fit_transform(train_csv['Label'])
train_csv.columns = ["ID", "Label"] + list(LABELS.values())
train_csv.head(5)

In [ ]:
label_count = train_csv.iloc[:, 2:].sum()
px.bar(label_count)

In [ ]:
for i in range(3):
    img_visual = TRAIN + '/' + train_csv['ID'].iloc[i]
    r = plt.imread(img_visual + '_red' + '.png')
    g = plt.imread(img_visual + '_green' + '.png')    
    b = plt.imread(img_visual + '_blue' + '.png')
    y = plt.imread(img_visual + '_yellow' + '.png')
    fig, ax = plt.subplots(1,4, figsize=(10,20))
    img = np.dstack((r, g, b, y))
    ax[0].set_title('r_g_b_y')
    ax[0].imshow(img)
    ax[0].axis('off')

    img = np.dstack((r, g, b))
    ax[1].set_title('r_g_b')
    ax[1].imshow(img)
    ax[1].axis('off')

    img = np.dstack((r, y, b))
    ax[2].set_title('r_y_b')
    ax[2].imshow(img)
    ax[2].axis('off')

    img = np.dstack((b, y, g))
    ax[3].set_title("b_y_g")
    ax[3].imshow(img)
    ax[3].axis('off')

    plt.show()

In [ ]:
# Input: list of image filters as png
# Output: list of image filters as np.arrays
def image_to_arrays(path):
    
    image_arrays = list()
    for image in path:
        array = np.asarray(Image.open(image))
        image_arrays.append(array)
        
    return image_arrays

In [ ]:
# Get single image that blends all RGBY into RGB
# Introduce the images as arrays. Can use the function above.

def get_blended_image(images): 
    # get rgby images for sample

    # blend rgby images into single array
    blended_array = np.stack(images[:-1], 2)

    # Create PIL Image
    blended_image = Image.fromarray( np.uint8(blended_array) )
    return blended_image

In [ ]:
# Introduce list of image filters
# Returns a processed image ready for the CNN and an encoded label as tensor
def image_prep(paths, label):

    img = image_to_arrays(paths)
    size = np.shape(img[0])[0]
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    img = tf.reshape(img, (1, size, size, 3))
    img = tf.image.resize(img, IMG_SIZE)

    label = tf.strings.split(label, sep='|')
    label = tf.strings.to_number(label, out_type=tf.int32)
    label = tf.reduce_sum(tf.one_hot(indices=label, depth=19), axis=0)
    label = tf.reshape(label, (1, 19))
    
    return img, label

In [ ]:
def apply_augmentation(image, label):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image], Tout=tf.float32)
    aug_img.set_shape((IMG_SIZE[0], IMG_SIZE[0], 3))
    
    return aug_img, label

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
titles = ['microtubules', 'nuclei', 'endoplasmic reticulum', 'protein of interest']
fig, axs = plt.subplots(3, 4, figsize =(16,8))
for entry in range(3):
    for channel in range(4):
        img = plt.imread(paths[entry][channel])
        axs[entry, channel].imshow(img)        
        if entry == 0:
            axs[0, channel].set_title(titles[channel])

In [ ]:
NUC_MODEL = "./nuclei-model.pth"
CELL_MODEL = "./cell-model.pth"
segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device="cuda",
    padding=False,
    multi_channel_model=True,
)

image = paths[4]
arrays = image_to_arrays(image)
nuclei = arrays[1]
cell = arrays[:-1]

# Nuclei segmentation
nuc_segmentations = segmentator.pred_nuclei([nuclei])

f, ax = plt.subplots(1, 2, figsize=(16,16))
ax[0].imshow(arrays[1])
ax[0].set_title('Original Nucleis', size=20)
ax[1].imshow(nuc_segmentations[0])
ax[1].set_title('Segmented Nucleis', size=20)
plt.show()

# Cell segmentation
inter_step = [[i] for i in image[:-1]]
cell_segmentations = segmentator.pred_cells(inter_step)

f, ax = plt.subplots(1, 2, figsize=(16,16))
ax[0].imshow(get_blended_image(arrays))
ax[0].set_title('Original Cells', size=20)
ax[1].imshow(cell_segmentations[0])
ax[1].set_title('Segmented Cells', size=20)
plt.show()

In [ ]:
# Nuclei mask
nuclei_mask = label_nuclei(nuc_segmentations[0])
# Cell masks
cell_nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])
# Plotting
f, ax = plt.subplots(1, 3, figsize=(16,16))
ax[0].imshow(nuclei_mask)
ax[0].set_title('Nuclei Mask', size=20)
ax[1].imshow(cell_nuclei_mask)
ax[1].set_title('Cell Nuclei Mask', size=20)
ax[2].imshow(cell_mask)
ax[2].set_title('Cell Mask', size=20)
plt.show()

In [ ]:
# Let's stack the original image and the segmentation mask, to see how the segmentation worked out
plt.figure(figsize=(20,20))
plt.imshow(get_blended_image(arrays))
plt.imshow(cell_mask, alpha=0.5)
plt.title('Segmentation results', size=40)
plt.axis('off')
plt.show()

In [ ]:
# Unique vector of cell_mask numbers
numbers = set(np.ravel(cell_mask))
numbers.remove(0)

fig = plt.figure(figsize=(25,8*len(numbers)/4))
index = 1

ax = fig.add_subplot(len(numbers)//4+1, 4, index)
ax.set_title("Complete Cell Mask", size=16)
plt.imshow(cell_mask)

index += 1
for number in numbers:
    isolated_cell = np.where(cell_mask==number, cell_mask, 0)
    ax = fig.add_subplot(len(numbers)//4+1, 4, index)
    ax.set_title(f"Segment {number}", size=16)
    plt.imshow(isolated_cell)
    index += 1

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Model
import wandb

In [ ]:
# We'll use EfficientNetB0 model, which requires an image dimension of (224,224,3).Therefor, we can only pass a 3 filter image... 
#We'll put aside the yellow filter for now.
IMG_SIZE = [224, 224]
BATCH_SIZE = 64
AUTOTUNE = tf.data.experimental.AUTOTUNE

colours = ['_red.png', '_blue.png', '_green.png']
TRAIN = '../input/hpa-single-cell-image-classification/train'
paths = [[os.path.join(TRAIN, train.iloc[idx,0])+ colour for colour in colours] for idx in range(len(train))]

In [ ]:
# Processing the data for training:
training_data = []
for i,path in enumerate(paths[:500]):
    img, label = image_prep(path, train['Label'][i])
    training_data.append([img,label])

train_ds = tf.data.Dataset.from_tensor_slices(([training_data[i][0] for i in range(len(training_data))], [training_data[i][1] for i in range(len(training_data))]))
len(train_ds)

In [ ]:
val_data = []
start_img = 500
val_num = 100
for i,path in enumerate(paths[start_img:start_img+val_num]):
    img, label = image_prep(path, train['Label'][i+start_img])
    val_data.append([img,label])

val_ds = tf.data.Dataset.from_tensor_slices(([val_data[i][0] for i in range(len(val_data))], [val_data[i][1] for i in range(len(val_data))]))

In [ ]:
base_model = EfficientNetB0(include_top=False, weights='imagenet')
base_model.trainable = True

inputs = layers.Input((IMG_SIZE[0], IMG_SIZE[0], 3))

x = base_model(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(LABELS), activation='sigmoid')(x)

tf.keras.backend.clear_session()

model = Model(inputs, outputs)
model.summary()

In [ ]:
tf.keras.backend.clear_session()

earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, verbose=0, mode='min',
    restore_best_weights=True
)

model.compile('adam', 'binary_crossentropy', metrics=[tf.keras.metrics.AUC(multi_label=True)])
#model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
#run = wandb.init(entity='ayush-thakur', project='hpa', job_type='train')

hist = model.fit(train_ds, 
          epochs=50,
          validation_data=val_ds,
          verbose=1,
          callbacks=[earlystopper]
                )
#plot_hist(hist)
#run.finish()